In [1]:
import gymnasium as gym


from stable_baselines3 import PPO, A2C
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from gymnasium.utils.env_checker import check_env


import gym_anytrading
from gym_anytrading.envs.trading_env import Actions
from gym_anytrading.envs.trading_env import Positions
import numpy as np



#env_id = "CartPole-v1"
env_id = "future-v0"

env = gym.make(env_id, frame_bound=(500, 1000), window_size = 20)
check_env(env)

shape is 182
p-1: 4210.0, p: 4210.0,  po: Positions.Empty,  po_reward: 0.0


/Users/liuzhe/miniconda3/envs/torch_env/lib/python3.9/site-packages/gymnasium/utils/env_checker.py:277: UserWarning: WARN: The environment (<OrderEnforcing<PassiveEnvChecker<FutureEnv<future-v0>>>>) is different from the unwrapped version (<FutureEnv<future-v0>>). This could effect the environment checker as the environment most likely has a wrapper applied to it. We recommend using the raw environment for `check_env` using `env.unwrapped`.
  logger.warn(
/Users/liuzhe/miniconda3/envs/torch_env/lib/python3.9/site-packages/gymnasium/utils/env_checker.py:221: UserWarning: WARN: A Box observation space minimum value is -infinity. This is probably too low.
  logger.warn(
/Users/liuzhe/miniconda3/envs/torch_env/lib/python3.9/site-packages/gymnasium/utils/env_checker.py:225: UserWarning: WARN: A Box observation space maximum value is -infinity. This is probably too high.
  logger.warn(
/Users/liuzhe/miniconda3/envs/torch_env/lib/python3.9/site-packages/gymnasium/utils/env_checker.py:186: Use

In [2]:
env.df.head()
#import pandas
#df = pandas.read_parquet("../gym_anytrading/datasets/data/RB_TICK.tick")
#df.columns

,symbol,tradingDay,updateTime,updateMillisec,lastPrice,volume,turnover,askPrice1,askPrice2,askPrice3,...,preDelta,average,diff,av1,av2,av3,bv1,bv2,bv3,pec
636,rb2305,20230307,09:00:00,500,4204.0,339857,1.430642e+10,4204.0,4205.0,4208.0,...,1.797693e+308,42095.407421,NaN,1.716003,1.041393,1.431364,2.671173,2.727541,2.727541,-0.017260
980,rb2305,20230307,09:00:01,0,4206.0,340295,1.432484e+10,4207.0,4209.0,4210.0,...,1.797693e+308,42095.347684,2.0,0.778151,0.602060,1.707570,1.518514,1.255273,1.255273,-0.012945
1482,rb2305,20230307,09:00:01,500,4207.0,340591,1.433729e+10,4210.0,4211.0,4212.0,...,1.797693e+308,42095.323482,1.0,1.255273,2.352183,2.075547,1.908485,1.518514,1.518514,-0.010787
1957,rb2305,20230307,09:00:02,0,4206.0,340773,1.434494e+10,4206.0,4207.0,4208.0,...,1.797693e+308,42095.308607,-1.0,1.301030,2.408240,1.301030,1.681241,1.826075,1.826075,-0.012945
2456,rb2305,20230307,09:00:02,500,4205.0,340909,1.435066e+10,4206.0,4207.0,4208.0,...,1.797693e+308,42095.292321,-1.0,1.113943,2.320146,1.204120,0.602060,1.875061,1.875061,-0.015102


In [3]:
import pysnooper
class Rsrs(object):
    def __init__(self, window, threshold):
        self._window = window
        self._threshold = threshold

    # https://durant35.github.io/2017/07/21/Algorithms_LeastSquaresLineFitting/

    def ols_a(self, x, y):
        t1, t2, t3, t4 = (0.0, 0.0, 0.0, 0.0)
        for i in range(len(x)):
            t1 = t1 + x[i]*x[i]
            t2 = t2 + x[i]
            t3 = t3 + x[i]*y[i]
            t4 = t4 + y[i]
        return (t3 * len(x) - t2*t4) / (t1 * len(x) - t2*t2)

    def single_ols_a(self, y):
        x = range(len(y))
        return self.ols_a(x, y)


    def predict(self, obs, **kargs):
        t1 = []
        index = 8
        position_v = obs[-2]

        while index < len(obs):
            t1.append(obs[index])
            index = index + 9
        m = self.single_ols_a(t1)
        if m > self._threshold and position_v != Positions.Long.value:
            ac = Actions.Buy
        elif m < - self._threshold and position_v != Positions.Short.value:
            ac = Actions.Sell
        else:
            ac = Actions.Hold

        return (np.array(ac.value), False)
    
    def mock(self):
        obs, info = env.reset()
        for i in range(20000):
            if i > 50:
                break
            # else
            action, _ = self.predict(obs, deterministic=False)
            obs, reward, done, trunced, info = env.step(action)
            if done or trunced:
                print("DONE---------")
                env.reset()
                break
            else:
                if info['ac'] != Actions.Hold:
                    #print(info)
                    pass
    


对于一个数列来说：
- 所有的值增加或减少一个常数，不影响斜率 => 所以可以直接用原始值，不需要减一个 base 什么的
- 改变两点间的宽度，会影响斜率 => 但是实际使用时是通过上下两个域值来用的，不同的间隔对应不同的值也就可以，所以时间其实随意用一个也就可以的
- 如何数列特别长，最后的变化几乎不会对斜率有什么影响，不符合策略希望的场景的定义，所以数列长度不需要太多
- 

In [4]:
x = [1,2,3,4,5,6,7]
y = [7,6,5,4,3,2,1]
k = [1,1,1,1,1,1,1]


In [5]:
model = Rsrs(20, 0.01)
model.mock()


return Sell, p_value: 2.0, short_value: 0
p-1: 4210.0, p: 4210.0,  po: Positions.Empty,  po_reward: 0.0
p-1: 4210.0, p: 4210.0,  po: Positions.Short,  po_reward: -0.0
return BUY, p_value: 0.0, long_value: 1
p-1: 4210.0, p: 4210.0,  po: Positions.Short,  po_reward: -0.0
return BUY, p_value: 2.0, long_value: 1
p-1: 4210.0, p: 4210.0,  po: Positions.Empty,  po_reward: 0.0
p-1: 4210.0, p: 4210.0,  po: Positions.Long,  po_reward: 0.0
p-1: 4210.0, p: 4210.0,  po: Positions.Long,  po_reward: 0.0
p-1: 4210.0, p: 4211.0,  po: Positions.Long,  po_reward: 10.0
p-1: 4211.0, p: 4210.0,  po: Positions.Long,  po_reward: -10.0
p-1: 4210.0, p: 4211.0,  po: Positions.Long,  po_reward: 10.0
p-1: 4211.0, p: 4211.0,  po: Positions.Long,  po_reward: 0.0
p-1: 4211.0, p: 4210.0,  po: Positions.Long,  po_reward: -10.0
p-1: 4210.0, p: 4211.0,  po: Positions.Long,  po_reward: 10.0
p-1: 4211.0, p: 4210.0,  po: Positions.Long,  po_reward: -10.0
p-1: 4210.0, p: 4210.0,  po: Positions.Long,  po_reward: 0.0
p-1: 4210